In [3]:
import pandas as pd
import json

def process_group_bump_data(file_path, output_json):
    cols = ['iyear', 'gname', 'targtype1_txt']
    df = pd.read_csv(file_path, encoding='ISO-8859-1', usecols=cols)
    group_mapping = {
        'ISIL': 'Islamic State of Iraq and the Levant (ISIL)',
        'taliban': 'Taliban',
        'SL': 'Shining Path (SL)'
    }
    output_data = {}
    for key, gtd_name in group_mapping.items():
        group_df = df[df['gname'] == gtd_name].copy()
        if group_df.empty:
            output_data[key] = {"timeline": []}
            continue
        top_targets = group_df['targtype1_txt'].value_counts().head(5).index.tolist()
        group_df = group_df[group_df['targtype1_txt'].isin(top_targets)]
        group_df['bin_start'] = (group_df['iyear'] // 3) * 3
        counts = group_df.groupby(['bin_start', 'targtype1_txt']).size().reset_index(name='count')
        pivot = counts.pivot(index='bin_start', columns='targtype1_txt', values='count').fillna(0)
        timeline = []
        for bin_start, row in pivot.iterrows():
            total_in_bin = row.sum()
            if total_in_bin == 0:
                continue
            values_dict = {}
            sort_helper = []
            for target in top_targets:
                val = row.get(target, 0)
                norm_val = (val / total_in_bin) * 100
                values_dict[target] = round(norm_val, 2)
                sort_helper.append((target, norm_val))
            sort_helper.sort(key=lambda x: x[1], reverse=True)
            order_list = [item[0] for item in sort_helper]
            label = f"{bin_start}-{bin_start+2}"
            timeline.append({"label": label, "values": values_dict, "order": order_list})
        output_data[key] = {
            "config": {"ribbonPadding": 2},
            "timeline": timeline,
            "top_targets": top_targets
        }
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2)

process_group_bump_data('terrorism_dataset.csv', 'groups_bump_data.json')